In [273]:
%matplotlib inline
import matplotlib.pyplot as plt
import artm
import copy


def print_and_gather_measures(model, metrics):
    print('Sparsity Phi: ', model.get_score('SparsityPhiScore').value)
    metrics.append(model.get_score('SparsityPhiScore').value)

    print('Sparsity Theta: ', model.get_score('SparsityThetaScore').value)
    metrics.append(model.get_score('SparsityThetaScore').value)

    print('Kernel contrast: ', model.get_score('TopicKernelScore').average_kernel_contrast)
    metrics.append(model.get_score('TopicKernelScore').average_kernel_contrast)

    print('Kernel purity: ', model.get_score('TopicKernelScore').average_kernel_purity)
    metrics.append(model.get_score('TopicKernelScore').average_kernel_purity)

    print('PerplexityScore: ', model.get_score('PerplexityScore').value)
    metrics.append(model.get_score('PerplexityScore').value)

    
def get_clustering(model, topics):
    phi_matrix = copy.deepcopy(model.phi_)
    pattern = '\(\'@default_class\', \'(.*?)\'\)'
    cleaned_index = [re.search(pattern, str(x)).group(1) for x in phi_matrix.index.tolist()]
    phi_matrix.index = cleaned_index
    num_topics = len(topics)
    clustering = [list() for _ in range(num_topics)]
    for i in range(num_topics):
        current_topic = topics[i]
        phi_matrix.sort_values(by = current_topic, ascending = False, inplace = True)
        top_tokens = phi_matrix.index[phi_matrix[current_topic] > 0]
        clustering[i] = top_tokens
    return clustering

def print_clustering(clustering, topics):
    num_topics = len(topics)
    for i in range(num_topics):
        print(topics[i])
        print(clustering[i])
        print(len(clustering[i]))
        
def save_clustering(clustering, topics, filename):
    f = open(filename, "w")
    num_topics = len(topics)
    for i in range(num_topics):
        f.write(topics[i])
        f.write("\n Amount of tags in cluster: ")
        f.write(str(len(clustering[i])))
        f.write("\n")
        for j in range(len(clustering[i])):
            f.write(clustering[i][j])
            f.write(", ")
        f.write("\n")
    f.close()
    
def print_top_tags(score_tracker):
    topics = score_tracker.topic_name
    tags = score_tracker.token
    weights = score_tracker.weight
    for i in range(1, len(topics) + 1):
        if ((topics[i-1] != topics[i]) | (i == 1)):
            print(topics[i])
        print('{}: {}, '.format(tags[i], weights[i]))
        
def save_top_tags(score_tracker, filename):
    f = open(filename, "w")
    topics = score_tracker.topic_name
    tags = score_tracker.token
    weights = score_tracker.weight
    for i in range(1, len(topics)):
        if ((topics[i-1] != topics[i]) | (i == 1)):
            f.write(topics[i])
            f.write("\n")
        f.write('{}: {}; \n '.format(tags[i], weights[i]))
    f.close()

# Smoke test

In [1]:
batch_vectorizer = artm.BatchVectorizer(data_path='vw.tags.100.txt', data_format='vowpal_wabbit',target_folder='posts-tags-100')
dictionary = batch_vectorizer.dictionary

topic_names = ['topic_{}'.format(i) for i in range(500)]
model_plsa = artm.ARTM(topic_names = topic_names, num_processors = 4,
                       scores = [artm.PerplexityScore(name = 'PerplexityScore', dictionary = dictionary), 
                                artm.SparsityPhiScore(name='SparsityPhiScore'),
                                artm.SparsityThetaScore(name='SparsityThetaScore'), 
                                artm.TopicKernelScore(name='TopicKernelScore', probability_mass_threshold=0.3),
                                artm.TopTokensScore(name='TopTokensScore', num_tokens = 20)],
                       seed = 62, show_progress_bars = True)


model_plsa.initialize(dictionary=dictionary)
model_plsa.fit_online(batch_vectorizer = batch_vectorizer, asynchronous = True)
model_plsa.dump_artm_model("model_plsa")

In [136]:
#dir(model_plsa.get_score('SparsityThetaScore'))
metrics_baseline = []
    
print_and_gather_measures(model_plsa, metrics_baseline)

Sparsity Phi:  0.9696954488754272
Sparsity Theta:  0.9932314157485962
Kernel contrast:  0.9987356662750244
Kernel purity:  0.9998822212219238
PerplexityScore:  12.941472053527832


##### $\Rightarrow$ conclusion: never assign values without specializing deepcopy


# Model 01 (PLSA, num_topics = 1000)

In [143]:
topic_names = ['topic_{}'.format(i) for i in range(1000)]
model_plsa = artm.ARTM(topic_names = topic_names, num_processors = 4,
                       scores = [artm.PerplexityScore(name = 'PerplexityScore', dictionary = dictionary), 
                                artm.SparsityPhiScore(name='SparsityPhiScore'),
                                artm.SparsityThetaScore(name='SparsityThetaScore'), 
                                artm.TopicKernelScore(name='TopicKernelScore', probability_mass_threshold = 0.1),
                                artm.TopTokensScore(name='TopTokensScore', num_tokens = 20)],
                       seed = 62, show_progress_bars = True)


model_plsa.initialize(dictionary=dictionary)

model_plsa.fit_online(batch_vectorizer = batch_vectorizer, asynchronous = True)
model_plsa.dump_artm_model("model_plsa_01")

Sparsity Phi:  0.9677442908287048
Sparsity Theta:  0.9959060549736023
Kernel contrast:  0.9970299005508423
Kernel purity:  0.9998293519020081
PerplexityScore:  9.101318359375


In [200]:
metrics_01 = []
print_and_gather_measures(model_plsa, metrics_01)

Sparsity Phi:  0.9677442908287048
Sparsity Theta:  0.9959060549736023
Kernel contrast:  0.9970299005508423
Kernel purity:  0.9998293519020081
PerplexityScore:  9.101318359375


In [235]:
clustering = get_clustering(model_plsa, topic_names)
#print_clustering(clustering, topic_names)
save_clustering(clustering, topic_names, "clustering_01.txt")

topic_0
Index(['materialize', 'ecmascript-5', 'parallax', 'localdate', 'gmp', 'maximo',
       'android-framelayout', 'yammer', 'boxing', 'pillow',
       ...
       'navmesh', 'intern', 'subsequence', 'boto3', 'android-volley',
       'powershell-5.0', 'blade', 'javascript-automation',
       'thinktecture-ident-server', 'powerpoint-2010'],
      dtype='object', length=963)
963
topic_1
Index(['gitlab', 'gitlab-ci', 'gitlab-ci-runner', 'javafx-8', 'alpine',
       'wso2-am', 'stringr', 'iis-8', 'arabic', 'gitlab-api',
       ...
       'google-now', 'bundling-and-minification', 'fuseki', 'retina',
       'lxml.html', 'azure-storage-files', 'shinydashboard', 'tomcat8',
       'openapi-generator', 'watch-os'],
      dtype='object', length=465)
465
topic_2
Index(['matplotlib-basemap', 'ssh-keys', 'install4j', 'csom', 'options',
       'probability-density', 'windowbuilder', 'django-filters', 'mongodump',
       'mongorestore',
       ...
       'meteor-helper', 'azure-cli', 'getopts', 'gr

      dtype='object', length=108)
108
topic_18
Index(['decorator', 'python-decorators', 'multiple-inheritance', 'body-parser',
       'metaclass', 'applepay', 'drone', 'functools', 'drone.io',
       'java-annotations',
       ...
       'byte-buddy', 'laravel-migrations', 'django-1.7', 'reactive-streams',
       'plotrix', 'sqlpackage', 'gitlab', 'kif', 'android-uiautomator', 'esi'],
      dtype='object', length=1538)
1538
topic_19
Index(['multer', 'h2o', 'octobercms', 'load', 'midi', 'throttling',
       'bandwidth', 'hexo', 'nedb', 'laravel-forge',
       ...
       'powermail', 'ember-cli', 'grunt-contrib-copy', 'azure-devops-rest-api',
       'ubuntu-13.10', 'homekit', 'fiware-cygnus', 'cloudera-cdh', 'ucwa',
       'mle'],
      dtype='object', length=948)
948
topic_20
Index(['ethereum', 'cpanel', 'solidity', 'dbus', 'whm', 'navigationview',
       'hal', 'intervention', 'airplay', 'android-navigationview',
       ...
       'magento-1.9.1', 'camanjs', 'fuseki', 'sktexture', 'gly

      dtype='object')
88
topic_31
Index(['apk', 'mnist', 'computational-geometry', 'fortify',
       'android-install-apk', 'dimensionality-reduction', 'keyvaluepair',
       'xiaomi', 'django-settings', 'sox',
       ...
       'simple-form-for', 'webdriver-io', 'bookshelf.js', 'knockout-3.0',
       'aws-php-sdk', 'ggvis', 'ubuntu-14.04', 'gulp', 'g++4.8', 'select2'],
      dtype='object', length=602)
602
topic_32
Index(['rectangles', 'avcapturesession', 'pygame-surface', 'fade', 'voltdb',
       'getattribute', 'android-package-managers', 'avcapturedevice',
       'traceroute', 'flashlight',
       ...
       'opshub', 'afnetworking', 'django-allauth', 'dart', 'kohana-3.2',
       'evil-mode', 'ng-map', 'webkit-transform', 'context-free-language',
       'caesar-cipher'],
      dtype='object', length=1581)
1581
topic_33
Index(['mariadb', 'wifi', 'pymysql', 'hotspot', 'wifimanager', 'wifi-direct',
       'stream-processing', 'wifip2p', 'rssi', 'ssid',
       ...
       'scichart', 'c

      dtype='object', length=420)
420
topic_50
Index(['email', 'smtp', 'telnet', 'webdav', 'verify', 'spam', 'forwarding',
       'mapi', 'eml', 'confirmation', 'bcc', 'cdo.message', 'mediatemple',
       'ejabberd', 'apple-mail', 'ldap', 'lucee', 'gsuite', 'exchange-server',
       'mandrill', 'pear', 'spf', 'php', 'starttls', 'email-address',
       'zend-mail', 'phpmailer', 'attachment', 'smtp-auth', 'mutt',
       'exchangewebservices'],
      dtype='object')
31
topic_51
Index(['html5-canvas', 'query-builder', 'ode', 'bootloader', 'paperjs',
       'kestrel', 'intel-fpga', 'get-childitem', 'grub', 'http-put',
       ...
       'hugo', 'webpack-4', 'lasagne', 'tumblr-themes', 'nunit-3.0',
       'protobuf-c', 'exoplayer', 'qt5.5', 'moviepy', 'spring-cloud'],
      dtype='object', length=203)
203
topic_52
Index(['haskell', 'functional-programming', 'f#', 'crystal-reports',
       'list-comprehension', 'immutability', 'monads', 'crosstab', 'typeclass',
       'sml', 'fold', 'designer'

      dtype='object', length=1468)
1468
topic_68
Index(['jaxb', 'marshalling', 'race-condition', 'pinvoke', 'unmarshalling',
       'tcplistener', 'dbnull', 'nettcpbinding', 'sta', 'automatic-properties',
       ...
       'capistrano3', 'jupyter', 'slimerjs', 'symfony-2.5', 'slick.js', 'pins',
       'gridextra', 'nsusernotification', 'compass', 'srcset'],
      dtype='object', length=1263)
1263
topic_69
Index(['random', 'subset', 'element', 'jpeg', 'shuffle', 'exif', 'fingerprint',
       'normal-distribution', 'random-seed', 'gd', 'biometrics',
       'gamma-distribution', 'seed', 'steganography', 'imagefilter', 'openvms',
       'random-sample', 'snapchat', 'gaussian', 'prng', 'druid',
       'mersenne-twister', 'ecdsa', 'gdlib', 'arules', 'srand', 'entropy',
       'dct'],
      dtype='object')
28
topic_70
Index(['full-text-search', 'feathersjs', 'full-text-indexing',
       'multiple-tables', 'laravel-facade', 'crate', 'oracle-text',
       'match-against', 'msvc12', 'highlightin

...

      dtype='object')
31
topic_904
Index(['rolling-computation', 'database-backups', 'facet-grid', 'umbraco7',
       'master-slave', 'chartkick', 'sha512', 'pearson-correlation',
       'laravel-eloquent', 'github-for-windows',
       ...
       'stm32f4discovery', 'jsonschema2pojo', 'xlconnect', 'skphysicsbody',
       'ibm-was', 'coldfusion-10', 'altbeacon', 'handsontable',
       'devise-invitable', 'react-component'],
      dtype='object', length=1604)
1604
topic_905
Index(['awk', 'sed', 'symlink', 'lcov', 'blank-line', 'hardlink', 'gawk',
       'unix', 'bash', 'cut', 'grep', 'shell', 'gcov', 'tr', 'nawk', 'ksh'],
      dtype='object')
16
topic_906
Index(['storage', 'onedrive', 'session-storage', 'phonegap-build', 'disk',
       'sharing', 'debian-jessie', 'corruption', 'cordova-ios', 'terrain',
       ...
       'ilnumerics', 'aws-rds', 'psutil', 'setcontentview',
       'sqlite-net-extensions', 'angularjs-validation', 'coreclr',
       'powershell-4.0', 'material-components-and

      dtype='object')
49
topic_923
Index(['gcc', 'linker', 'syntax-error', 'scanf', 'cygwin', 'gnu', 'ld', 'elf',
       'readline', 'startup', 'sqlcmd', 'libstdc++', 'gcov', 'linker-scripts',
       'weak', 'dwarf', 'relocation', 'g++', 'rhel7', 'fopen',
       'unresolved-external', 'gcc4.8', 'lcov', 'lto', 'compilation', 'c++',
       'gcc4.7', 'assimp', 'capl', 'gnu-toolchain', 'linux-kernel',
       'compiler-optimization', 'inline-assembly', 'inline-functions', 'sparc',
       'powerpc'],
      dtype='object')
36
topic_924
Index(['setstate', 'metal', 'echo', 'reminders', 'size-classes', 'xcode6',
       'java-5', 'speex', 'java1.4', 'phpexcelreader',
       ...
       'scalaz7', 'blade', 'cloudkit', 'coldfusion-10', 'mediainfo', 'xcode7',
       'ucanaccess', 'synology', 'bootstrap-modal', 'sobel'],
      dtype='object', length=1120)
1120
topic_925
Index(['statsmodels', 'navigator', 'cakephp-2.0', 'nmap', 'weighted',
       'oracle-fusion-middleware', 'contingency', 'gdbserver', 

      dtype='object', length=532)
532
topic_941
Index(['server', 'router', 'event-sourcing', 'tinkerpop', 'py2neo', 'linphone',
       'cycle', 'parametric-polymorphism', 'spray-json', 'livereload',
       ...
       'hogan.js', 'openshift-client-tools', 'boost-spirit-x3',
       'azure-powershell', 'gremlin', 'ansible', 'seaborn',
       'entity-framework-6.1', 'twitter-bootstrap-rails', 'crystal-lang'],
      dtype='object', length=1078)
1078
topic_942
Index(['instagram-api', 'iis-10', 'encapsulation', 'tail-recursion', 'friend',
       'restify', 'pdi', 'jsreport', 'libuv', 'cbc-mode',
       ...
       'kendo-window', 'vuforia', 'cleardb', 'fiddlercore', 'ng-hide',
       'dartium', 'hybris', 'regexp-replace', 'datomic', 'onelogin'],
      dtype='object', length=1617)
1617
topic_943
Index(['hibernate', 'java-ee', 'serial-port', 'stack-overflow', 'can-bus',
       'database-partitioning', 'capture', 'ejb-3.0', 'setattribute',
       'sessionfactory', 'uptime', 'ejb-2.x', 'hibernate-

      dtype='object', length=1113)
1113
topic_977
Index(['deployment', 'scp', 'timing', 'release-management', 'descriptor',
       'high-resolution', 'ibm-was', 'clickonce', 'strongloop',
       'microservices',
       ...
       'azure-servicebus-topics', 'allure', 'mimekit', 'webix',
       'react-proptypes', 'dm-script', 'dcos', 'homestead', 'pyspark',
       'nsurlsessiondownloadtask'],
      dtype='object', length=327)
327
topic_978
Index(['yum', 'fedora', 'package-managers', 'centos6', 'autoload', 'antivirus',
       'chain', 'virus', 'member-functions', 'flickr',
       ...
       'symfony-2.8', 'gulp-sourcemaps', 'symfony-2.5', 'magrittr',
       'github-desktop', 'spoon', 'odeint', 'buck', 'javascript-automation',
       'uwp-xaml'],
      dtype='object', length=957)
957
topic_979
Index(['unit-testing', 'mockito', 'mocking', 'powermock', 'stub', 'jmockit',
       'easymock', 'oracleclient', 'qunit', 'software-quality', 'isolation',
       'static-block', 'jmock', 'iprincipal',

      dtype='object', length=506)
506
topic_993
Index(['testcafe', 'html2canvas', 'coroutine', 'elasticsearch-aggregation',
       'arduino-ide', 'web-api-testing', 'ui-testing', 'pymc3', 'cgo',
       'systemjs',
       ...
       'knife', 'system.net.httpwebrequest', 'zapier', 'xcode-storyboard',
       'typo3-extensions', 'endpoints-proto-datastore', 'pgrouting', 'stan',
       'android-fileprovider', 'cross-validation'],
      dtype='object', length=1560)
1560
topic_994
Index(['wpf', 'xaml', 'binding', 'extension-methods', 'wpf-controls',
       'communication', 'dependency-properties', 'backgroundworker',
       'datatemplate', 'datatrigger', 'dispatcher', 'scrollviewer', 'blend',
       'expander', 'routed-events', 'expression-blend', 'c#',
       'windows-10-universal', 'scichart', 'devexpress-wpf', 'wpf-animation',
       'windows-8.1-universal', 'uwp-xaml', 'modern-ui', 'relativesource',
       'flipview', 'helix-3d-toolkit', 'catel', 'itemscontrol', 'wpf-style',
       'dockp

In [274]:
#print_top_tags(model_plsa.get_score('TopTokensScore'))
save_top_tags(model_plsa.get_score('TopTokensScore'), "top_tags_01.txt")